In [170]:

import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

from pyspark.sql import *
from pyspark.sql.functions import *

from pyspark.sql.types import StructType, StructField, IntegerType, StringType


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from src.utils import MyUtils

import os.path
from os import listdir
from os.path import isfile, join
import sys

import subprocess
import yaml
import sqlite3

if (sys.modules.get('src.logistic_regression') is not None): 
    del sys.modules['src.logistic_regression']
import src.logistic_regression 

if (sys.modules.get('src.people_generator') is not None): 
    del sys.modules['src.people_generator']
import src.people_generator

# Add the parent directory to the system path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

print(parent_dir)



/home/ho-yu


In [171]:

src.people_generator.generate_people_data(n = 20, percent_cylon = 0.25)

Generated 100 rows of data and saved to 'bsg_people_data.csv'
   id   first_name  last_name salutation call_sign         dob  weight_lbs  \
0   1  Christopher      Greer        Mr.            3677-04-22         123   
1   2         John     Harris        Ms.            3662-11-11         115   
2   3       Glenda     Wright        Mr.            3674-05-18         123   
3   4       Jeanne   Trujillo        Mr.            3662-10-08         176   
4   5       Audrey     Hunter        Dr.            3712-04-20         108   
5   6       Gerald       Dunn        Mr.            3712-02-06         130   
6   7      Stephen      Allen        Sir            3679-01-03         190   
7   8       Joshua  Rodriguez        Mr.            3739-05-29         114   
8   9        Brian   Martinez        Dr.            3667-12-20         248   
9  10         Ruth    Hensley        Sir            3689-12-25         138   

   height_m gender  dna_mutations_n home_colony_id  is_cylon  
0      1.58     

In [172]:
print("Current working directory set to:", os.getcwd())

# Read the config.yaml file
if os.path.exists('config.yaml'):
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)

if os.path.exists('../config.yaml'):
    with open('../config.yaml', 'r') as file:
        config = yaml.safe_load(file)

dir = config['fileio']['working_directory2']

print("dir: " + dir)

# Set the current working directory
os.chdir(dir)
print("Current working directory set to:", os.getcwd())

Current working directory set to: /home/ho-yu/bsg
dir: /home/ho-yu/bsg
Current working directory set to: /home/ho-yu/bsg


In [173]:
conf = SparkConf().setAppName("appName").setMaster("local")

try:
    sc.stop()
except:
    pass

sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


conn = sqlite3.connect(dir + "/db/simple.db")
cursor = conn.cursor()

25/02/16 19:57:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/16 19:57:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [174]:

bsg_cylons_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("model_number", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("gender", StringType(), True)
])

bsg_people = spark.read.csv(dir + "/csv/people.csv", header=True, inferSchema=True) \
    .withColumnRenamed("id", "person_id")

bsg_cylons = spark.read.csv(dir + "/csv/cylons.csv", header=True, schema = bsg_cylons_schema) \
    .withColumnRenamed("id", "cylon_id") \
    .withColumnRenamed("model_number", "cylons_model_number")

bsg_cylons_people = spark.read.csv(dir + "/csv/cylons_people.csv", header=True) \
    .withColumnRenamed("person_id", "cylons_people_person_id") \
    .withColumnRenamed("model_number", "cylons_people_model_number")


bsg_data_people_cylons = bsg_people \
    .join(bsg_cylons_people, bsg_people.person_id == bsg_cylons_people.cylons_people_person_id, 'left') \
    # .join(bsg_cylons, bsg_cylons_people.cylons_people_model_number == bsg_cylons.cylons_model_number) \


current_year = 3760
current_month = 4
current_day = 12
current_date = f"{current_year}-{current_month:02d}-{current_day:02d}"

query1 = f"""
    SELECT 
        weight_lbs, 
        height_m,
        CASE 
            WHEN gender = 'M' THEN 0 
            WHEN gender = 'F' THEN 1 
        END AS gender,
        YEAR(DATE('{current_date}')) - YEAR(dob) - 
            (CASE 
                WHEN MONTH(DATE('{current_date}')) < MONTH(dob) 
                        OR (MONTH(DATE('{current_date}')) = MONTH(dob) AND DAY(DATE('{current_date}')) < DAY(dob)) 
                THEN 1 ELSE 0 
                END) AS age,
            dna_mutations_n,    
        CASE 
            WHEN cylons_people_model_number IS NULL THEN -1 
            ELSE 1 
        END AS is_cylon
           
    FROM bsg_data_people_cylons 
    LIMIT 100
"""
query2 = "SELECT * FROM bsg_data_people_cylons LIMIT 100"

# bsg_data_people_cylons.createOrReplaceTempView("bsg_data_people_cylons")
# bsg_data_people_cylons = spark.sql(query2)
# print("bsg_data_people_cylons")
# bsg_data_people_cylons.show()

bsg_data_people_cylons.createOrReplaceTempView("bsg_data_people_cylons")
bsg_data_people_cylons = spark.sql(query1)
# print("bsg_data_people_cylons")
# bsg_data_people_cylons.show()

bsg_data_people_cylons.createOrReplaceTempView("bsg_data")
bsg_data = spark.sql("SELECT * FROM bsg_data")
bsg_data.show()

# print("bsg_people")
# bsg_people.show()

# print("bsg_cylons")
# bsg_cylons.show()

# print("bsg_cylons_people")
# bsg_cylons_people.show()

# print("bsg_data_people_cylons")
# bsg_data_people_cylons.show()

+----------+--------+------+---+---------------+--------+
|weight_lbs|height_m|gender|age|dna_mutations_n|is_cylon|
+----------+--------+------+---+---------------+--------+
|       155|    1.78|     0| 30|           1556|      -1|
|       200|    1.74|     0| 60|        5454544|      -1|
|       120|     1.6|     1| 24|          25444|      -1|
|       160|     1.7|     0| 44|           3566|      -1|
|       115|    1.55|     1| 55|      534354354|      -1|
|       155|     1.7|     0| 45|            150|       1|
|       116|    1.56|     1| 24|            188|       1|
|       116|    1.56|     1| 24|             90|       1|
|       170|     1.7|     0| 69|        7756664|      -1|
|       160|     1.8|     0| 39|             35|       1|
|       158|    1.75|     0| 32|           4415|      -1|
|       185|    1.85|     0| 39|          11321|      -1|
|       121|    1.59|     1| 35|          44556|      -1|
|       150|    1.68|     0| 34|           7548|      -1|
|       110|  

In [175]:
df_pandas = bsg_data.toPandas()

features =['weight_lbs', 'height_m', 'gender', 'age', 'dna_mutations_n']
features = ["gender", "age", "dna_mutations_n"]

X_train = df_pandas[features]
y_train = df_pandas[['is_cylon']]

X_train.to_csv("./data/X_train.csv", header=False, index=False)  # No header, no index
y_train.to_csv("./data/y_train.csv", header=False, index=False)

# X_train.to_csv("./data/X_test.csv", header=False, index=False)  # No header, no index
# y_train.to_csv("./data/y_test.csv", header=False, index=False)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print(X_train.head())  # Preview features
print(y_train.head())  # Preview target labels

X_train shape: (17, 3)
y_train shape: (17, 1)
   gender  age  dna_mutations_n
0       0   30             1556
1       0   60          5454544
2       1   24            25444
3       0   44             3566
4       1   55        534354354
   is_cylon
0        -1
1        -1
2        -1
3        -1
4        -1


In [176]:
query3 = f"""
    SELECT 
        weight_lbs, 
        height_m,
        CASE 
            WHEN gender = 'M' THEN 0 
            WHEN gender = 'F' THEN 1 
        END AS gender,
        YEAR(DATE('{current_date}')) - YEAR(dob) - 
            (CASE 
                WHEN MONTH(DATE('{current_date}')) < MONTH(dob) 
                        OR (MONTH(DATE('{current_date}')) = MONTH(dob) AND DAY(DATE('{current_date}')) < DAY(dob)) 
                THEN 1 ELSE 0 
                END) AS age,
            dna_mutations_n,    
        is_cylon
           
    FROM bsg_people_generated 
"""




bsg_people_generated = spark.read.csv(dir + "/data/bsg_people_data.csv", header=True, inferSchema=True) \
    .withColumnRenamed("id", "person_id")

bsg_people_generated.createOrReplaceTempView("bsg_people_generated")
bsg_people_generated = spark.sql(query3)
bsg_people_generated.show()

df_pandas_test = bsg_people_generated.toPandas()

# features = ['weight_lbs', 'height_m', 'gender', 'age', 'dna_mutations_n']
X_test = df_pandas_test[features]
y_test = df_pandas_test[['is_cylon']]

X_test.to_csv("./data/X_test.csv", header=False, index=False)  # No header, no index
y_test.to_csv("./data/y_test.csv", header=False, index=False)

# X_test.to_csv("./data/X_train.csv", header=False, index=False)  # No header, no index
# y_test.to_csv("./data/y_train.csv", header=False, index=False)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print(X_test.head())  # Preview features
print(y_test.head())  # Preview target labels

+----------+--------+------+---+---------------+--------+
|weight_lbs|height_m|gender|age|dna_mutations_n|is_cylon|
+----------+--------+------+---+---------------+--------+
|       123|    1.58|     1| 82|         140136|      -1|
|       115|    1.62|     1| 97|         628871|      -1|
|       123|    1.69|     0| 85|         227274|      -1|
|       176|    1.71|     0| 97|         387437|      -1|
|       108|    1.84|     0| 47|         266538|      -1|
|       130|    1.52|     1| 48|         475191|      -1|
|       190|    1.53|     1| 81|         635813|      -1|
|       114|    1.88|     0| 20|             93|       1|
|       248|    1.85|     0| 92|         287141|      -1|
|       138|    1.64|     1| 70|         325604|      -1|
|       220|    1.53|     0| 10|            929|      -1|
|       169|    1.89|     0| 53|         267049|      -1|
|       196|    1.63|     0| 46|         504640|      -1|
|       246|    1.59|     1| 21|            874|      -1|
|       115|  

In [177]:
# READ in data
# df_X_train = pd.read_csv('data/X_train.csv', header=None)
# df_y_train = pd.read_csv('data/y_train.csv', header=None)

df_X_test = pd.read_csv('data/X_test.csv', header=None)
df_y_test = pd.read_csv('data/y_test.csv', header=None)

# save in numpy arrays
X_train = df_X_train.to_numpy()
y_train = df_y_train.to_numpy()
X_test = df_X_test.to_numpy()
y_test = df_y_test.to_numpy()

# get training set size
n_train = X_train.shape[0]

# normalize all features to [0,1] or [-1,1]
X_all = MyUtils.normalize_neg1_pos1(np.concatenate((X_train, X_test), axis=0))


X_train = X_all[:n_train]
X_test = X_all[n_train:]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print()
print(X_train)

(20, 3)
(20, 1)
(20, 3)
(20, 1)

[[-1.          0.14606742  0.62737798]
 [-1.         -0.52808989  0.40498065]
 [ 1.          0.57303371  0.29961418]
 [-1.         -0.59550562 -0.3483731 ]
 [-1.         -0.59550562 -0.08230035]
 [ 1.         -0.84269663 -0.99817547]
 [-1.          0.86516854 -0.70790886]
 [ 1.         -0.50561798 -0.21684354]
 [-1.         -0.30337079 -0.17517195]
 [ 1.         -0.7752809  -0.99832332]
 [-1.         -0.28089888 -0.75231408]
 [ 1.          0.41573034 -0.99982069]
 [ 1.         -0.61797753 -0.9994841 ]
 [-1.         -0.95505618 -0.99898707]
 [-1.          0.05617978  0.80781433]
 [ 1.         -0.23595506  0.81799079]
 [-1.          0.23595506 -0.99959105]
 [-1.          0.07865169 -0.93983777]
 [ 1.         -0.91011236 -0.99995596]
 [ 1.         -0.3258427  -0.9999245 ]]
